<a href="https://colab.research.google.com/github/rca-devs/CIS-5450/blob/main/CIS_5450_course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://esap.seas.upenn.edu/wp-content/themes/summer_engineering/media/seas_logo.png)

#Introduction

Welcome to our course project for CIS-5450 (Big Data Analytics)!

Our team consists of:

*   Ryan Abbate (MSE-DS, inaugural cohort)
*   Paul Hamel (MCIT, Spring 2022 cohort)
*   Milan Mistry (MCIT, Fall 2021 cohort)

<br>
For this project, we built machine learning models that predict a borrower's propensity to default on a loan. Throughout this notebook, we walk through our end-to-end process, which included:

* data ingestion
* exploratory data analysis (EDA)
* data wrangling
* data cleaning
* machine learning modeling
* data visualization


# Motivation

Individuals with insufficient or non-existent credit histories often encounter challenges in securing loans.  Those fortunate enough to obtain loans may fall victim to unscrupulous lenders who impose exorbitant interest rates and unfavorable lending terms.

Lenders generally aim to optimize profits while minimizing potential risks.  Lenders that use *accurate* predictive models when assessing default risk among certain customer profiles are able to extend credit to borrowers who may not meet traditional lending standards.  This data-driven approach ensures that borrowers who are capable of loan repayment receive the capital that they need, which has a stimulative effect on economic activity.


# Data Source

For this project, we used loan repayment data from a Kaggle competition hosted by [Home Credit Group](https://www.homecredit.net/) in 2018.  Details about the competition may be found [here](https://www.kaggle.com/competitions/home-credit-default-risk/overview).  The data consists of nine (9) csv files totaling approximately 3GB in size.  

# Data Ingestion

To retrieve and ingest the data from Kaggle's public API, users must authenticate with an API token.  To create an API token, please follow the instructions outlined [here](https://www.kaggle.com/competitions/home-credit-default-risk/overview).  The API token will be in the form of a JSON file named `kaggle.json`.  

#Dependencies

Let's begin by installing the required dependencies.


In [1]:
# install dependencies with silenced output
!pip install kaggle --quiet
!pip install tqdm --quiet

Next, we import the Python libraries that we'll use for this project.

In [2]:
# import required libraries
from google.colab import drive
import os
import pandas as pd
from tqdm.notebook import tqdm


Run the cell below, then click "Choose Files".  Browse to the `kaggle.json` file, then select this file for upload.

In [3]:
# upload your Kaggle API key to download the data
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ryanabbate","key":"09d24706541b17b088d1497a650c71d8"}'}

Next, we'll create a Kaggle directory and copy the file containing our API token into the newly created directory.

In [4]:
# create the Kaggle directory
!mkdir ~/.kaggle

# copy the file containing API key to newly created directory and change file permissions
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

We'll now download the dataset in the form of a zip file.

In [5]:
# download the dataset
!!kaggle competitions download -c home-credit-default-risk

['Downloading home-credit-default-risk.zip to /content',
 '',
 '  0% 0.00/688M [00:00<?, ?B/s]',
 '  1% 5.00M/688M [00:00<00:14, 49.7MB/s]',
 '  2% 17.0M/688M [00:00<00:11, 59.9MB/s]',
 '  4% 28.0M/688M [00:00<00:08, 78.4MB/s]',
 '  6% 41.0M/688M [00:00<00:07, 86.2MB/s]',
 '  7% 50.0M/688M [00:00<00:09, 73.8MB/s]',
 '  9% 65.0M/688M [00:00<00:07, 91.5MB/s]',
 ' 13% 91.0M/688M [00:00<00:04, 139MB/s] ',
 ' 16% 107M/688M [00:01<00:04, 146MB/s] ',
 ' 18% 122M/688M [00:01<00:04, 137MB/s]',
 ' 20% 139M/688M [00:01<00:03, 146MB/s]',
 ' 23% 159M/688M [00:01<00:03, 163MB/s]',
 ' 26% 176M/688M [00:01<00:03, 155MB/s]',
 ' 28% 193M/688M [00:01<00:04, 128MB/s]',
 ' 31% 216M/688M [00:01<00:03, 155MB/s]',
 ' 35% 241M/688M [00:01<00:02, 179MB/s]',
 ' 38% 260M/688M [00:02<00:02, 179MB/s]',
 ' 41% 279M/688M [00:02<00:02, 184MB/s]',
 ' 43% 298M/688M [00:02<00:02, 182MB/s]',
 ' 46% 316M/688M [00:02<00:02, 183MB/s]',
 ' 50% 345M/688M [00:02<00:01, 215MB/s]',
 ' 53% 366M/688M [00:05<00:12, 26.4MB/s]',
 ' 56

After downloading the dataset, we'll unzip the file into our `Content` directory.

In [6]:
# unzip the zip file into the Content directory
!unzip /content/home-credit-default-risk.zip

Archive:  /content/home-credit-default-risk.zip
  inflating: HomeCredit_columns_description.csv  
  inflating: POS_CASH_balance.csv    
  inflating: application_test.csv    
  inflating: application_train.csv   
  inflating: bureau.csv              
  inflating: bureau_balance.csv      
  inflating: credit_card_balance.csv  
  inflating: installments_payments.csv  
  inflating: previous_application.csv  
  inflating: sample_submission.csv   


Now, we'll identify the files that we'll use during our analysis and exclude those that are irrelevant.

In [7]:
# the list of files to exclude from relevant data files
excluded_files = ['kaggle.json', 'home-credit-default-risk.zip', '.config', 'sample_data', 'sample_submission.csv', 'HomeCredit_columns_description.csv']

# create a list of relevant data files
data_files = [data_file for data_file in os.listdir('/content') if data_file not in excluded_files]
data_files


['credit_card_balance.csv',
 'application_train.csv',
 'POS_CASH_balance.csv',
 'bureau.csv',
 'application_test.csv',
 'bureau_balance.csv',
 'installments_payments.csv',
 'previous_application.csv']

After identifying the relevant eight (8) data files, we'll create a Pandas dataframe for each.  Notice that we create an empty dictionary `d`, then iterate through the list of relevant file and append `_df` to the file name when assigning the name of the dataframe.  We use the popular `tqdm` library to monitor progress as the dataframes are being created.

In [8]:
# iterate over list of relevant data files and create a Pandas dataframe for each

d = {}

for data_file in tqdm(data_files):
  print('Creating:', data_file.removesuffix('.csv') + '_df')
  d[data_file.removesuffix('.csv') + '_df'] = pd.read_csv(data_file)


  0%|          | 0/8 [00:00<?, ?it/s]

Creating: credit_card_balance_df
Creating: application_train_df
Creating: POS_CASH_balance_df
Creating: bureau_df
Creating: application_test_df
Creating: bureau_balance_df
Creating: installments_payments_df
Creating: previous_application_df


Since we stored the dataframes in a dictionary `d`, we access a given dataframe by its key.  Let's inspect the data.

In [9]:
# access dataframes via a dictionary such as d['bureau_df']
d['bureau_df'].head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [18]:
for key, df in d.items():
  print(key)
  print('\n')
  print(df.head())
  print('\n')


credit_card_balance_df


   SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  AMT_BALANCE  \
0     2562384      378907              -6       56.970   
1     2582071      363914              -1    63975.555   
2     1740877      371185              -7    31815.225   
3     1389973      337855              -4   236572.110   
4     1891521      126868              -1   453919.455   

   AMT_CREDIT_LIMIT_ACTUAL  AMT_DRAWINGS_ATM_CURRENT  AMT_DRAWINGS_CURRENT  \
0                   135000                       0.0                 877.5   
1                    45000                    2250.0                2250.0   
2                   450000                       0.0                   0.0   
3                   225000                    2250.0                2250.0   
4                   450000                       0.0               11547.0   

   AMT_DRAWINGS_OTHER_CURRENT  AMT_DRAWINGS_POS_CURRENT  \
0                         0.0                     877.5   
1                         0.0          